In [1]:
from __future__ import division
from __future__ import print_function

import json

from utils import features_construct_graph, spatial_construct_graph
import torch
import os
import argparse
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from config import Config
import anndata as ad
from matplotlib.image import imread
from pathlib import Path
from scipy.sparse import csr_matrix

/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = 'MOSTA'
config_file = '../ini/MOSTA.ini'
config = Config(config_file)
highly_genes, k, k1 = config.fdim, config.k , config.radius
savepath = "../generate_data/" + dataset + "/"
if not os.path.exists(savepath):
    os.mkdir(savepath)

In [3]:
def normalize(adata, highly_genes=3000):
    print("start select HVGs")
    # sc.pp.filter_cells(adata, min_genes=100)
    sc.pp.filter_genes(adata, min_cells=50)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=highly_genes)
    adata = adata[:, adata.var['highly_variable']].copy()
    return adata
    
adata = sc.read_h5ad('./E9.5_E1S1.MOSTA.h5ad')
adata.X = adata.X.toarray()

adata = normalize(adata, highly_genes=highly_genes)

barcodes = np.array(adata.obs.index)
names = np.array(adata.var.index)

n = len(adata.obsm['spatial'])


fadj = features_construct_graph(adata.X, k=k)
sadj, graph_nei, graph_neg = spatial_construct_graph(adata.obsm["spatial"], k=k1)

ground = adata.obs['annotation']
ground_class = adata.obs['annotation'].unique()
num_list = [f'{x}' for x in range(len(ground_class))]
for x,y in zip(ground_class,num_list):
    ground.replace(x, y, inplace=True)
adata.obs['ground_truth'] = ground

adata.obsm["fadj"] = fadj
adata.obsm["sadj"] = sadj
adata.obsm["graph_nei"] = graph_nei.numpy()
adata.obsm["graph_neg"] = graph_neg.numpy()

print(adata)
adata.write(savepath + 'DMGCN.h5ad')
print('done')

start select HVGs


/root/miniconda3/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


start features construct graph
start spatial construct graph
AnnData object with n_obs × n_vars = 5913 × 3000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'annotation', 'Regulon - 2310011J03Rik', 'Regulon - 5730507C01Rik', 'Regulon - Alx1', 'Regulon - Alx3', 'Regulon - Alx4', 'Regulon - Ar', 'Regulon - Arid3a', 'Regulon - Arid3c', 'Regulon - Arnt2', 'Regulon - Arx', 'Regulon - Ascl1', 'Regulon - Atf1', 'Regulon - Atf4', 'Regulon - Atf5', 'Regulon - Atf6', 'Regulon - Atf7', 'Regulon - Bach1', 'Regulon - Bach2', 'Regulon - Barhl1', 'Regulon - Barx1', 'Regulon - Batf', 'Regulon - Bcl11a', 'Regulon - Bcl3', 'Regulon - Bcl6', 'Regulon - Bcl6b', 'Regulon - Bclaf1', 'Regulon - Bdp1', 'Regulon - Bhlha15', 'Regulon - Bhlhe22', 'Regulon - Bhlhe23', 'Regulon - Bhlhe41', 'Regulon - Bmyc', 'Regulon - Boll', 'Regulon - Bptf', 'Regulon - Brca1', 'Regulon - Brf1', 'Regulon - Brf2', 'Regulon - Bsx', 'Regulon - Cdx1', 'Regulon - Cdx2', 'Regulon - Cebpa',